Idea explanation (Spanish):
https://youtu.be/1jRmrLFTtKo

Code explanation (Spanish):
https://youtu.be/JLFqlf-Yh84


In [1]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
# Loading your IBM Q account(s)
provider = IBMQ.load_account()

/opt/conda/lib/python3.7/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


In [2]:
!pip install -I git+https://github.com/qiskit-community/IBMQuantumChallenge2020-Grader.git

  Cloning https://github.com/qiskit-community/IBMQuantumChallenge2020-Grader.git to /tmp/pip-req-build-7vxb7j0h
  Using cached numpy-1.19.4-cp37-cp37m-manylinux2010_x86_64.whl (14.5 MB)
Processing ./.cache/pip/wheels/bc/b7/b0/9099f1685d1bed7d4f5a7ee189ab41d3fb71ae44d71ac05994/qiskit-0.23.1-py3-none-any.whl
  Using cached requests-2.25.0-py2.py3-none-any.whl (61 kB)
  Using cached qiskit_terra-0.16.1-cp37-cp37m-manylinux2010_x86_64.whl (8.5 MB)
  Using cached qiskit_aqua-0.8.1-py3-none-any.whl (2.1 MB)
  Using cached qiskit_ibmq_provider-0.11.1-py3-none-any.whl (195 kB)
  Using cached qiskit_aer-0.7.1-cp37-cp37m-manylinux2010_x86_64.whl (17.5 MB)
  Using cached qiskit_ignis-0.5.1-py3-none-any.whl (204 kB)
  Using cached urllib3-1.26.2-py2.py3-none-any.whl (136 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Using cached certifi-2020.12.5-py2.py3-none-any.whl (147 kB)
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
  Using cached ply-3.11-py2.py3-none-any.whl (49 k

In [2]:
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller
from qiskit.compiler import transpile

def coste(circuit):
    #function to get the cost of a circuit
    pass_ = Unroller(['u3', 'cx'])
    pm = PassManager(pass_)
    new_circuit = pm.run(circuit)
    print(new_circuit.count_ops())
    return new_circuit.count_ops()['u3'] + new_circuit.count_ops()['cx'] * 10

Algorithm idea:

We are going to convert the board into a bit string in which I write a 1 if the position has an asteroid.
Example:

1     0     0     0


0     1     1     0


1     0     0      0      


1     0     0      1

= |1 0 0 0 0 1 1 0 1 0 0 0 1 0 0 1>

Basically, to create the qRAM I used the hamming distance to determinate the orden in which I introduced the address and after that I applied simplifications to the multi-crontol gates.

The most interesting part of the program is the oracle. Let's understand the idea:
Firstly, we are going to work only with 4 asteroid. The trivial solution that we know that is unsolvable is the diagonal:

0     0     0     1


0     0     1     0


0     1     0      0      


1     0     0      0


So if our original board with 6 asteriods has got four asteroids in the diagonal, is unsolvable.
The key of the solution comes here, all the posible boards with four asteroids that are not solvable, are a permutation of rows and columns of the diagonal board (because in that way, always we'll have 1 asteroid in each column and row), so instead of define all the posible unsolvable boards, I can define a basis of posible permutations between rows and columns to guarantee that I'll get all this unsolvable combinations. Well, actually I thought in the opposite way, my idea was apply my basis of permutations to the original board and check if in one of this combinations that we have got is the diagonal with ones. It worked but we could optimize much more. One important point is that given an unsolvable board with 4 asteroids is equivalent permutate the colums i and j to the rows i and j, so it's enought for the problem, if we only apply permutations to the columns and check if the diagonal has got 4 ones. Finally, in order to optimize a bit more the solution, I realized that I could reduce the cost by balancing the number of permutations and the number of initial boards (apart from the diagonal)
When scaling the problem, I recommend using the diagonal and its lateral displacements. It means that we'll be able to generate the n! unsolvable boards (n is the side of the board) with n trivial initial boards.



In [16]:
def week3_ans_func(problem_set):
    
    
    def convertir_input(datos):
        # this function translate the input into the bit string
        solucion = []
        
        for fila in datos:
            asteroides = [0] * 16
            for dato in fila:
                asteroides[4 * int(dato[0]) + int(dato[1])] = 1
            solucion.append(asteroides)
            
        return solucion
    
    datos_preparados = convertir_input(problem_set)
    iniciales = datos_preparados.copy()

    def transformar_a_distintos(datos):
        # Instead of insert the input directly in the qRAM,
        # We insert the different between the current data and the previous data
        
        solucion = [datos[0]]
        
        for i in range(len(datos)-1):
            nuevo_termino = []
            for j in range(len(datos[0])):
                if datos[i][j] == datos[i+1][j]:
                    nuevo_termino.append(0)
                else:
                    nuevo_termino.append(1)
            solucion.append(nuevo_termino)
        return solucion
    

    def distancia_hamming(a, b):
        # return the hamming distance between two vectors
        d = 0
        for i,j in zip(a,b):
            if i != j: d += 1
                
        return d
    
    def matriz_coste():
        # return the cost matrix where the nodes are the address and the cost the hamming distance
        matriz = []
        for i in range(len(datos_preparados)):
            fila = []
            for j in range(len(datos_preparados)):
                if i != j:
                    fila.append(distancia_hamming(bin(32 + i)[-4:],bin(32 + j)[-4:]))
                else:
                    fila.append(99999)
            matriz.append(fila)
        return matriz
    
    matriz = matriz_coste()
    

    
    def algoritmo_voraz(nodo_inicial):
        # return the path that we'll use using the nearest neighbourd
        ruta = []
        ruta.append(nodo_inicial)

        nodo_actual = nodo_inicial
        
        while len(ruta) < len(datos_preparados):
            nodo_actual = min_index(matriz[nodo_actual], ruta)
            ruta.append(nodo_actual)
        return ruta
    
    def min_index(array, array2):
        #auxiliar function to use the previous function
        minimo = 99999
        for i, j in enumerate(array):
            if j < minimo and i not in array2:
                solucion = i
                minimo = j
        return solucion
    
    
    orden_datos = algoritmo_voraz(12)
    
    datos_preparados = transformar_a_distintos([datos_preparados[i] for i in orden_datos])
    
    Us = []
    for i in range(len(datos_preparados)):
        Us.append([bin(32 + i)[-4:] for i in orden_datos[i:]])
    

    
    
    def simplificar_tupla(tupla):
        # with this function we can simplify the multicontrol gates
        # notation: "1" means the control is activated if the value is 1
        #           "0" means the control is activated if the value is 0
        #           "2" means there is not control in this position
        # example:
        # qc.x(0)  qc.ccx(0,1,A)    qc.x(0)  => "0122"  
        
        tupla_actual = tupla.copy()
        nueva_tupla = []
        while tupla_actual != nueva_tupla and tupla_actual != []:
            nueva_tupla = []
            
            for i, num1 in enumerate(tupla_actual):
                
                aux2 = 0
                if str(type(num1)) != "<class 'int'>":
                    for j, num2 in enumerate(tupla_actual[i+1:]):
                        if str(type(num2)) != "<class 'int'>":
                            d, indice = digitos_diferentes(num1,num2)
                            if d == 1 and aux2 == 0:
                                aux = list(num1)
                                aux[indice] = '2'
                                aux = "".join(aux)

                                nueva_tupla.append(aux)
                                tupla_actual[i] = -i - 1
                                tupla_actual[j+i+1] = -j-i - 1

                                aux2 = 1

                    if aux2 == 0: nueva_tupla.append(num1)
            tupla_actual = nueva_tupla
        
        return nueva_tupla
        
    
    def multi_simplificacion(tupla):
        # with this function we use recursively the previous simplification
        actual = tupla.copy()
        nueva = simplificar_tupla(actual)
        if actual == nueva:
            return nueva
        else:
            return multi_simplificacion(nueva)
       
    
    def digitos_diferentes(num1, num2):
        # auxiliar, count the number of different elements in two strings
        d = 0
        indice = -1
        for i in range(len(num1)):
            if num1[i] != num2[i]:
                d += 1
                indice = i
        return d, indice

        
        
    # In this part we are defining the gates that we'll use
    Us2 = []
    for tupla in Us:
        Us2.append(multi_simplificacion(tupla))
        

    puertas = []
    dic_puertas = {}
    for indx, u in enumerate(Us2):
        for c in u:
            if c not in dic_puertas.keys():
                dic_puertas[c] = []
            for ind, k in enumerate(datos_preparados[indx]):
                if k == 1:

                    dic_puertas[c].append(str(ind + 4))   
        
    nuevo_dic = {}
    for indx in dic_puertas:
        dic_puertas[indx] = sorted(dic_puertas[indx])
        
        for i in range(len(dic_puertas[indx])-1):
            if dic_puertas[indx][i] == dic_puertas[indx][i+1]:
                dic_puertas[indx][i] = -1
                dic_puertas[indx][i+1] = -1
                
        nuevo_dic[indx] = []
        for i in dic_puertas[indx]:
            if i != -1:
                nuevo_dic[indx].append(i)
                
    def qRAM2(dic): 
        # I don't use this qRAM, I define other later but this is my general qRAM
        # The other qRAM has been created manually
        
        qc = QuantumCircuit(28,4)
        ancillas = [26,27]
        for indice in dic:
            objetivos = []
            for i in dic[indice]:
                objetivos.append(int(i))
                
            if indice.count('2') == 0: 
                controls = []
                for indx, i in enumerate(indice):
                    if i != "2":
                        controls.append(int(indx))
                        if i == "0":
                            qc.x(indx)
                            
                qc.ccx(controls[0], controls[1], ancillas[0])
                qc.ccx(controls[2], ancillas[0], ancillas[1])
                for obj in objetivos:
                    qc.ccx(controls[3], ancillas[1], obj)
                qc.ccx(controls[2], ancillas[0], ancillas[1])
                qc.ccx(controls[0], controls[1], ancillas[0])
                
                for indx, i in enumerate(indice):
                    if i == "0":
                        qc.x(indx)
                       
            
            elif indice.count('2') == 1:
                controls = []
                for indx, i in enumerate(indice):
                    if i != "2":
                        controls.append(int(indx))
                        if i == "0":
                            qc.x(indx)
                            
                qc.ccx(controls[0], controls[1], ancillas[0])
                for obj in objetivos:
                    qc.ccx(controls[2], ancillas[0], obj)
                qc.ccx(controls[0], controls[1], ancillas[0])
                
                for indx, i in enumerate(indice):
                    if i == "0":
                        qc.x(indx)
                       
            elif indice.count('2') == 2: 
                controls = []
                for indx, i in enumerate(indice):
                    if i != "2":
                        controls.append(int(indx))
                        if i == "0":
                            qc.x(indx)
                            
                
                for obj in objetivos:
                    qc.ccx(controls[0], controls[1], obj)
              
                
                for indx, i in enumerate(indice):
                    if i == "0":
                        qc.x(indx)
                        
            elif indice.count('2') == 3:
                controls = []
                for indx, i in enumerate(indice):
                    if i != "2":
                        controls.append(int(indx))
                        if i == "0":
                            qc.x(indx)
                qc.cx(controls[0], objetivos)   
                for indx, i in enumerate(indice):
                    if i == "0":
                        qc.x(indx)
                
            elif indice.count('2') == 4:
                qc.x(objetivos)
                
        return qc
    
    
    def qRAM_casera(dic):
        # My homemade qRAM
        
        qc = QuantumCircuit(28,4)
        ancillas = [25,26,27]
        
        qc.ccx(0,2,ancillas[0])
        for l in dic['1212']:
            qc.cx(ancillas[0], int(l))
        qc.ccx(1, ancillas[0], ancillas[1])
        for l in dic['1112']:
            qc.cx(ancillas[1], int(l))
            
    
        qc.x(3)
        qc.ccx(3,ancillas[1],ancillas[2])
        for l in dic['1110']:
            qc.cx(ancillas[2], int(l))
        qc.ccx(3,ancillas[1],ancillas[2])  
        qc.x(3)
        
        qc.ccx(1, ancillas[0], ancillas[1])
        qc.ccx(3, ancillas[0], ancillas[1])
        
        for l in dic['1211']:
            qc.cx(ancillas[1], int(l))
            
        qc.ccx(3, ancillas[0], ancillas[1])
      
   
        qc.x(0)
        qc.cx(2,ancillas[0])
        
        for l in dic['0212']:
            qc.cx(ancillas[0], int(l))
        qc.x(3)
        qc.ccx(3, ancillas[0], ancillas[1])
        for l in dic['0210']:
            qc.cx(ancillas[1], int(l))
        qc.ccx(3,ancillas[0], ancillas[1])
        qc.x(3)
        qc.ccx(0,2,ancillas[0])
        qc.x(0)

        
        qc.x(1)
        qc.ccx(0,1,ancillas[0])
        for l in dic['1022']:
            qc.cx(ancillas[0], int(l))
        qc.x(3)
        qc.ccx(3,ancillas[0],ancillas[1])
        for l in dic['1020']:
            qc.cx(ancillas[1], int(l))
        qc.ccx(3,ancillas[0],ancillas[1])
        qc.x(3)

        
        qc.cx(1, ancillas[0])
        qc.cx(0, ancillas[0])
        qc.x([0,1,ancillas[0]])

        qc.ccx(3,ancillas[0],ancillas[1])
        for l in dic['0121']:
            qc.cx(ancillas[1], int(l))
        qc.ccx(3,ancillas[0],ancillas[1])
        qc.ccx(0,1,ancillas[0])
        qc.x(0)

        qc.x(2)
        qc.ccx(1,2, ancillas[0])
        qc.ccx(3,ancillas[0],ancillas[1])
        for l in dic['2101']:
            qc.cx(ancillas[1], int(l))
        qc.ccx(3,ancillas[0], ancillas[1])

        qc.x(2)
        qc.cx(1, ancillas[0])
        
        for l in dic['2112']:
            qc.cx(ancillas[0], int(l))
        qc.ccx(1,2,ancillas[0])

        qc.ccx(0, 3, ancillas[0])
        for l in dic['1221']:
            qc.cx(ancillas[0], int(l))
        qc.ccx(0,3,ancillas[0])
        
        qc.x(0)
        for l in dic['0222']:
            qc.cx(0, int(l))
        qc.x(0)

        qc.x(1)
        for l in dic['2022']:
            qc.cx(1, int(l))
        qc.x(1)

        for l in dic['2221']:
            qc.cx(3, int(l))

        for l in dic['2222']:
            qc.x(int(l))
  
        return qc
        
  # so now we'll start with the Oracle

    qc = QuantumCircuit(28,4)
    qc.h(range(4))
    qc.h(range(20,22))
    for cont in (range(1)):
        
        qc += qRAM_casera(nuevo_dic)
        
        
        # if qubit 20 is 1 we permutate columns 0 and 1
        # if qubit 21 is 1 we permutate columns 2 and 3
        # We get this permutation with the fredkin gate that it is control swap

        
        qc.fredkin(20,4,5)       
        qc.fredkin(20,8,9)
        qc.fredkin(20,12,13)
        qc.fredkin(20,16,17)


        qc.fredkin(21,7,6)
        qc.fredkin(21,11,10)
        qc.fredkin(21,15,14)
        qc.fredkin(21,19,18)

        # this are the boards that are going to generate all the posible unsolvable 
        # boards with our permutation set
        qc.h(16)
        qc.mct([7,10,13],16,26, mode = "basic")

        qc.mct([9,6,15],16,26, mode = "basic")

        qc.mct([5,14,11],16,26, mode = "basic")
        qc.h(16)
        
        qc.h(18)
        qc.mct([12,9,7],18,26, mode = "basic")
        
        qc.h(19)
        qc.mct([5,12,10],19,26, mode = "basic")
        qc.h(19)
        
        qc.mct([8,5,15],18,26, mode = "basic")
        qc.h(18)

          
        qc.fredkin(21,19,18)
        qc.fredkin(21,15,14)
        qc.fredkin(21,11,10)
        qc.fredkin(21,7,6)
       

        qc.fredkin(20,16,17)
        qc.fredkin(20,12,13)
        qc.fredkin(20,8,9)
        qc.fredkin(20,4,5)           
        
        # uncomputing the auxiliar qubits
        qc += qRAM_casera(nuevo_dic)

        # Grover gate
        qc.h(range(4))
        qc.h(20)
        qc.h(21)
        qc.x(range(4))
        qc.x(21)
        qc.x(20)
        qc.h(3)
        qc.mct([0,1,2,20,21], 3, [23,24,22], mode = "basic")
        qc.h(3)
        qc.x(range(4))
        qc.x(21)
        qc.x(20)
        qc.h(range(4))
        qc.h(21)
        qc.h(20)
    
    qc.measure(range(4),range(4))
    
    return qc


So now, let's see the results

In [17]:
from qc_grader import grade_ex3, prepare_ex3, submit_ex3

# Execute your circuit with following prepare_ex3() function.
# The prepare_ex3() function works like the execute() function with only QuantumCircuit as an argument.
job = prepare_ex3(week3_ans_func)

result = job.result()
counts = result.get_counts()
original_problem_set_counts = counts[0]

original_problem_set_counts

Running week3_ans_func...
Computing cost...
Starting experiments. Please wait...
You may monitor the job (id: 5fce4d15fe5c6b001429b7ae) status and proceed to grading when it successfully completes.


{'0000': 63,
 '0001': 48,
 '0010': 43,
 '0011': 64,
 '0100': 59,
 '0101': 159,
 '0110': 57,
 '0111': 61,
 '1000': 55,
 '1001': 50,
 '1010': 63,
 '1011': 48,
 '1100': 46,
 '1101': 69,
 '1110': 64,
 '1111': 51}

In [18]:
grade_ex3(job)

Grading your answer. Please wait...

Congratulations 🎉! Your answer is correct.
Your score is 7799.
The lower your score the better!
Feel free to submit your answer.


In [19]:
problem_set = [[['0', '1'], ['0', '2'], ['1', '0'], ['2', '0'], ['3', '1'], ['3', '3']],
 [['0', '2'], ['0', '3'], ['1', '1'], ['1', '3'], ['2', '0'], ['2', '1']],
 [['0', '0'], ['0', '3'], ['2', '1'], ['2', '2'], ['3', '0'], ['3', '1']],
 [['0', '0'], ['0', '1'], ['0', '2'], ['1', '1'], ['2', '0'], ['3', '2']],
 [['0', '1'], ['1', '2'], ['1', '3'], ['2', '0'], ['3', '0'], ['3', '1']],
 [['0', '2'], ['0', '3'], ['1', '1'], ['2', '0'], ['2', '1'], ['3', '0']],
 [['0', '0'], ['0', '3'], ['1', '2'], ['2', '0'], ['2', '1'], ['3', '3']],
 [['0', '2'], ['1', '1'], ['1', '3'], ['2', '0'], ['2', '3'], ['3', '2']],
 [['0', '1'], ['0', '3'], ['2', '0'], ['2', '2'], ['3', '0'], ['3', '3']],
 [['0', '0'], ['0', '2'], ['1', '0'], ['2', '2'], ['2', '3'], ['3', '3']],
 [['1', '0'], ['1', '3'], ['2', '1'], ['2', '2'], ['3', '2'], ['3', '3']],
 [['0', '0'], ['1', '0'], ['2', '1'], ['2', '2'], ['3', '2'], ['3', '3']],
 [['0', '0'], ['1', '1'], ['1', '2'], ['2', '1'], ['2', '3'], ['3', '0']],
 [['0', '1'], ['0', '3'], ['2', '1'], ['2', '2'], ['3', '0'], ['3', '1']],
 [['0', '0'], ['0', '1'], ['1', '1'], ['1', '3'], ['3', '2'], ['3', '3']],
 [['0', '0'], ['0', '3'], ['1', '2'], ['1', '3'], ['3', '0'], ['3', '1']]]


# 0110


qc = week3_ans_func(problem_set)
backend = Aer.get_backend("qasm_simulator")
job2 = execute(qc, backend, shots = 1000)
job2.result().get_counts()

{'0000': 68,
 '0001': 53,
 '0010': 59,
 '0011': 47,
 '0100': 57,
 '0101': 47,
 '0110': 167,
 '0111': 50,
 '1000': 54,
 '1001': 53,
 '1010': 62,
 '1011': 45,
 '1100': 61,
 '1101': 61,
 '1110': 64,
 '1111': 52}

In [20]:
problem_set = \
    [[['0', '2'], ['1', '0'], ['1', '2'], ['1', '3'], ['2', '0'], ['3', '3']],
    [['0', '0'], ['0', '1'], ['1', '2'], ['2', '2'], ['3', '0'], ['3', '3']],
    [['0', '0'], ['1', '1'], ['1', '3'], ['2', '0'], ['3', '2'], ['3', '3']],
    [['0', '0'], ['0', '1'], ['1', '1'], ['1', '3'], ['3', '2'], ['3', '3']],
    [['0', '2'], ['1', '0'], ['1', '3'], ['2', '0'], ['3', '2'], ['3', '3']],
    [['1', '1'], ['1', '2'], ['2', '0'], ['2', '1'], ['3', '1'], ['3', '3']],
    [['0', '2'], ['0', '3'], ['1', '2'], ['2', '0'], ['2', '1'], ['3', '3']],
    [['0', '0'], ['0', '3'], ['1', '2'], ['2', '2'], ['2', '3'], ['3', '0']],
    [['0', '3'], ['1', '1'], ['1', '2'], ['2', '0'], ['2', '1'], ['3', '3']],
    [['0', '0'], ['0', '1'], ['1', '3'], ['2', '1'], ['2', '3'], ['3', '0']],
    [['0', '1'], ['0', '3'], ['1', '2'], ['1', '3'], ['2', '0'], ['3', '2']],
    [['0', '0'], ['1', '3'], ['2', '0'], ['2', '1'], ['2', '3'], ['3', '1']],
    [['0', '1'], ['0', '2'], ['1', '0'], ['1', '2'], ['2', '2'], ['2', '3']],
    [['0', '3'], ['1', '0'], ['1', '3'], ['2', '1'], ['2', '2'], ['3', '0']],
    [['0', '2'], ['0', '3'], ['1', '2'], ['2', '3'], ['3', '0'], ['3', '1']],
    [['0', '1'], ['1', '0'], ['1', '2'], ['2', '2'], ['3', '0'], ['3', '1']]]


# 0101


qc = week3_ans_func(problem_set)
backend = Aer.get_backend("qasm_simulator")
job2 = execute(qc, backend, shots = 1000)
job2.result().get_counts()

{'0000': 50,
 '0001': 59,
 '0010': 62,
 '0011': 47,
 '0100': 60,
 '0101': 187,
 '0110': 43,
 '0111': 57,
 '1000': 60,
 '1001': 49,
 '1010': 50,
 '1011': 63,
 '1100': 65,
 '1101': 48,
 '1110': 43,
 '1111': 57}

In [21]:
problem_set = [[['0', '2'], ['0', '3'], ['1', '1'], ['2', '0'], ['3', '0'], ['3', '1']],
 [['0', '1'], ['0', '3'], ['2', '0'], ['2', '2'], ['3', '0'], ['3', '1']],
 [['0', '0'], ['0', '3'], ['1', '1'], ['1', '3'], ['2', '2'], ['2', '3']],
 [['0', '2'], ['0', '3'], ['1', '0'], ['1', '1'], ['2', '3'], ['3', '1']],
 [['0', '1'], ['0', '2'], ['1', '0'], ['2', '0'], ['2', '3'], ['3', '3']],
 [['0', '3'], ['1', '0'], ['1', '2'], ['2', '1'], ['2', '2'], ['3', '3']],
 [['0', '1'], ['0', '3'], ['2', '0'], ['2', '3'], ['3', '2'], ['3', '3']],
 [['1', '0'], ['1', '1'], ['2', '1'], ['2', '3'], ['3', '2'], ['3', '3']],
 [['0', '1'], ['0', '2'], ['1', '0'], ['1', '3'], ['2', '3'], ['3', '0']],
 [['0', '0'], ['1', '1'], ['1', '3'], ['2', '0'], ['3', '2'], ['3', '3']],
 [['0', '1'], ['0', '2'], ['1', '3'], ['2', '0'], ['3', '0'], ['3', '3']],
 [['0', '0'], ['0', '1'], ['2', '0'], ['2', '3'], ['3', '2'], ['3', '3']],
 [['0', '3'], ['1', '0'], ['1', '2'], ['2', '2'], ['3', '0'], ['3', '3']],
 [['0', '0'], ['0', '3'], ['1', '0'], ['1', '1'], ['2', '0'], ['2', '2']],
 [['0', '1'], ['0', '3'], ['1', '2'], ['2', '0'], ['2', '1'], ['3', '2']],
 [['0', '2'], ['0', '3'], ['1', '3'], ['2', '0'], ['2', '2'], ['3', '0']]]


# 1100


qc = week3_ans_func(problem_set)
backend = Aer.get_backend("qasm_simulator")
job2 = execute(qc, backend, shots = 1000)
job2.result().get_counts()

{'0000': 74,
 '0001': 53,
 '0010': 52,
 '0011': 50,
 '0100': 53,
 '0101': 59,
 '0110': 54,
 '0111': 49,
 '1000': 61,
 '1001': 47,
 '1010': 52,
 '1011': 53,
 '1100': 180,
 '1101': 51,
 '1110': 52,
 '1111': 60}

In [22]:
problem_set =[[['0', '0'], ['0', '2'], ['1', '0'], ['1', '1'], ['3', '1'], ['3', '3']],
 [['0', '2'], ['0', '3'], ['1', '1'], ['1', '3'], ['2', '0'], ['2', '1']],
 [['0', '0'], ['1', '0'], ['2', '1'], ['2', '3'], ['3', '2'], ['3', '3']],
 [['0', '2'], ['0', '3'], ['1', '1'], ['1', '2'], ['3', '0'], ['3', '2']],
 [['0', '2'], ['0', '3'], ['2', '0'], ['2', '1'], ['3', '1'], ['3', '3']],
 [['0', '1'], ['0', '3'], ['1', '2'], ['1', '3'], ['2', '2'], ['3', '1']],
 [['0', '0'], ['1', '0'], ['2', '2'], ['2', '3'], ['3', '1'], ['3', '3']],
 [['0', '0'], ['0', '1'], ['1', '2'], ['2', '0'], ['3', '1'], ['3', '2']],
 [['0', '1'], ['0', '2'], ['1', '0'], ['1', '3'], ['3', '0'], ['3', '1']],
 [['0', '0'], ['0', '2'], ['1', '0'], ['1', '3'], ['2', '1'], ['2', '2']],
 [['0', '0'], ['0', '1'], ['0', '3'], ['1', '0'], ['2', '1'], ['3', '3']],
 [['0', '0'], ['0', '3'], ['1', '1'], ['1', '3'], ['2', '0'], ['2', '2']],
 [['0', '1'], ['1', '3'], ['2', '0'], ['2', '1'], ['2', '3'], ['3', '0']],
 [['0', '0'], ['1', '1'], ['2', '0'], ['2', '3'], ['3', '1'], ['3', '2']],
 [['0', '0'], ['0', '3'], ['1', '2'], ['2', '2'], ['3', '1'], ['3', '3']],
 [['0', '2'], ['0', '3'], ['1', '0'], ['1', '2'], ['2', '1'], ['2', '2']]]


# 1011


qc = week3_ans_func(problem_set)
backend = Aer.get_backend("qasm_simulator")
job2 = execute(qc, backend, shots = 1000)
job2.result().get_counts()

{'0000': 55,
 '0001': 39,
 '0010': 50,
 '0011': 55,
 '0100': 53,
 '0101': 56,
 '0110': 54,
 '0111': 63,
 '1000': 52,
 '1001': 59,
 '1010': 51,
 '1011': 194,
 '1100': 52,
 '1101': 58,
 '1110': 47,
 '1111': 62}

In [23]:
    problem_set =[[['0', '1'], ['0', '2'], ['1', '0'], ['2', '1'], ['3', '0'], ['3', '2']],
     [['0', '0'], ['1', '1'], ['1', '2'], ['2', '1'], ['3', '0'], ['3', '2']],
     [['0', '1'], ['1', '2'], ['1', '3'], ['2', '1'], ['3', '0'], ['3', '3']],
     [['0', '0'], ['0', '2'], ['1', '1'], ['1', '2'], ['2', '0'], ['3', '1']],
     [['0', '0'], ['0', '2'], ['1', '1'], ['1', '2'], ['2', '1'], ['2', '3']],
     [['1', '1'], ['1', '2'], ['2', '0'], ['2', '3'], ['3', '1'], ['3', '3']],
     [['1', '1'], ['1', '3'], ['2', '2'], ['2', '3'], ['3', '0'], ['3', '2']],
     [['0', '1'], ['0', '2'], ['1', '2'], ['2', '0'], ['2', '3'], ['3', '1']],
     [['0', '0'], ['0', '3'], ['1', '1'], ['2', '2'], ['2', '3'], ['3', '1']],
     [['0', '1'], ['0', '2'], ['1', '3'], ['2', '0'], ['2', '2'], ['3', '3']],
     [['0', '2'], ['0', '3'], ['1', '2'], ['2', '1'], ['2', '3'], ['3', '1']],
     [['0', '1'], ['0', '3'], ['1', '0'], ['1', '2'], ['3', '0'], ['3', '1']],
     [['0', '0'], ['0', '3'], ['1', '2'], ['1', '3'], ['2', '0'], ['2', '1']],
     [['0', '0'], ['1', '3'], ['2', '0'], ['2', '1'], ['2', '2'], ['3', '2']],
     [['0', '0'], ['0', '1'], ['0', '2'], ['1', '1'], ['2', '2'], ['3', '0']],
     [['0', '3'], ['1', '0'], ['1', '2'], ['2', '1'], ['3', '1'], ['3', '3']]]
    
# 1011
qc = week3_ans_func(problem_set)
backend = Aer.get_backend("qasm_simulator")
job2 = execute(qc, backend, shots = 1000)
job2.result().get_counts()

{'0000': 48,
 '0001': 49,
 '0010': 60,
 '0011': 62,
 '0100': 58,
 '0101': 51,
 '0110': 49,
 '0111': 59,
 '1000': 63,
 '1001': 52,
 '1010': 72,
 '1011': 160,
 '1100': 55,
 '1101': 52,
 '1110': 55,
 '1111': 55}